In [1]:
import pandas as pd
from tqdm import tqdm
import re
from datetime import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [2]:
## 인코더
class Encoder(nn.Module):

    def __init__(self, input_size=4096, hidden_size=1024, num_layers=2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,
        #                     dropout=0.1, bidirectional=False)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.1, bidirectional=False)

    def forward(self, x):
        output, hidden = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        return hidden
    
## 디코더
class Decoder(nn.Module):

    def __init__(self, input_size=4096, hidden_size=1024, output_size=4096, num_layers=2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,
#                             dropout=0.1, bidirectional=False)
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.1, bidirectional=False)

        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden):
        output, hidden = self.lstm(x, hidden)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        prediction = self.fc(output)

        return prediction, hidden
    
## RNN Auto Encoder
class RNNAutoEncoder(nn.Module):

    def __init__(self,
                 input_dim: int,
                 latent_dim: int,
                 window_size: int=1,
                 **kwargs) -> None:
        """
        :param input_dim: 변수 Tag 갯수
        :param latent_dim: 최종 압축할 차원 크기
        :param window_size: 길이
        :param kwargs:
        """

        super(RNNAutoEncoder, self).__init__()

        self.latent_dim = latent_dim
        self.input_dim = input_dim
        self.window_size = window_size

        if "num_layers" in kwargs:
            num_layers = kwargs.pop("num_layers")
        else:
            num_layers = 1

        self.encoder = Encoder(
            input_size=input_dim,
            hidden_size=latent_dim,
            num_layers=num_layers,
        )
        self.reconstruct_decoder = Decoder(
            input_size=input_dim,
            output_size=input_dim,
            hidden_size=latent_dim,
            num_layers=num_layers,
        )

    def forward(self, src:torch.Tensor, **kwargs):
        batch_size, sequence_length, var_length = src.size()

        ## Encoder 넣기
        encoder_hidden = self.encoder(src)
        
        inv_idx = torch.arange(sequence_length - 1, -1, -1).long()
        reconstruct_output = []
        temp_input = torch.zeros((batch_size, 1, var_length), dtype=torch.float).to(src.device)
        hidden = encoder_hidden
        for t in range(sequence_length):
            temp_input, hidden = self.reconstruct_decoder(temp_input, hidden)
            reconstruct_output.append(temp_input)
        reconstruct_output = torch.cat(reconstruct_output, dim=1)[:, inv_idx, :]
        
        return [reconstruct_output, src]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        recons = args[0]
        input = args[1]
        
        ## MSE loss(Mean squared Error)
        loss =F.mse_loss(recons, input)
        return loss

In [3]:
from dataset import CurrentDataset

In [4]:
path = '/home/workspace/iot_ai_model/dataset/current/train/**/normal/*.csv'
dataset = CurrentDataset(path)

In [5]:
len(dataset)

68100

In [6]:
model = RNNAutoEncoder(3, 500, num_layers=2).cuda()

In [7]:
trn_dataloader = DataLoader(dataset,
                            batch_size=64,
                            shuffle=True,
                            num_workers=4,
                            pin_memory=True,
                            drop_last=False)

In [8]:
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = F.mse_loss

In [ ]:
model.train()
hist_loss = []
pre_loss = 1.

In [10]:
for epoch in range(epochs):
    progress = tqdm(trn_dataloader)
    cost = 0.
    for path, cur, target in progress:
        cur = cur.cuda()
        output, _ = model(cur)
        loss = criterion(cur, output)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        progress.set_postfix_str(f'loss {loss.item()}')
        
        cost += loss.item()
    avg_loss = cost / len(train_loader)
    hist_loss.append(avg_loss)    
    progress.set_postfix_str(f'loss {avg_loss}')    

 32%|███▏      | 394/1217 [11:03<23:06,  1.68s/it, loss 98.58082580566406] 


KeyboardInterrupt: 

In [11]:
import numpy as np

In [13]:
np.random.random((1,300,300,3))

array([[[[0.08704378, 0.37334953, 0.7192736 ],
         [0.25851468, 0.08333939, 0.91786107],
         [0.61416842, 0.71758518, 0.56501066],
         ...,
         [0.18090796, 0.07895112, 0.93121052],
         [0.39633596, 0.10610315, 0.63402062],
         [0.31739749, 0.42837523, 0.71957548]],

        [[0.79688461, 0.81463725, 0.81951224],
         [0.43310532, 0.72843846, 0.3233063 ],
         [0.84818989, 0.36250963, 0.97919309],
         ...,
         [0.98370834, 0.01058058, 0.74192041],
         [0.57554875, 0.05285465, 0.73751619],
         [0.99109348, 0.69378585, 0.75760908]],

        [[0.57404126, 0.81415867, 0.01540538],
         [0.29621688, 0.64454995, 0.93091213],
         [0.80338385, 0.98140393, 0.24922374],
         ...,
         [0.00865711, 0.61437302, 0.94842847],
         [0.36159889, 0.27436632, 0.94589492],
         [0.75465225, 0.00206451, 0.67654211]],

        ...,

        [[0.24829576, 0.36673569, 0.04645826],
         [0.73450276, 0.58413347, 0.91377221]